In [1]:
import urllib2
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import time
import random

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]


In [2]:
df_of_companies = pd.read_csv('Company_list_final.csv', index_col=0) # < This should be the stock symbol list of companies we are interested in.
df_of_companies.columns = ['Symbol']
list_of_companies = [x for x in df_of_companies.Symbol] + ['GM']
list_of_companies


['DDD',
 'MMM',
 'ABT',
 'ANF',
 'ACN',
 'BHVN',
 'APB',
 'ATVI',
 'AYI',
 'ATGE',
 'ADTN',
 'AAP',
 'AMD',
 'ACM',
 'AES',
 'AET',
 'AMG',
 'AFL',
 'AGCO',
 'AEM',
 'AGU',
 'AL',
 'ALK',
 'ALB',
 'AA',
 'ALEX',
 'ARE',
 'Y',
 'AKP',
 'LNT',
 'AHPI',
 'MDRX',
 'ALL',
 'ALLY',
 'MO',
 'AMCX',
 'DOX',
 'UHAL',
 'AAL',
 'AAT',
 'AEO',
 'AXP',
 'AMP',
 'ABC',
 'AMGN',
 'AFSI-PA',
 'AEUA',
 'ADI',
 'NLY',
 'ANSS',
 'ANTM',
 'APA',
 'ARI',
 'AIY',
 'APLE',
 'AMAT',
 'ATR',
 'WTR',
 'ARMK',
 'ARH-PC.CL',
 'ADM',
 'AWI',
 'HRS',
 'ARW',
 'AJG',
 'APAM',
 'ASNA',
 'ASH',
 'ASB',
 'AGO',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AVY',
 'CAR',
 'AVT',
 'BHGE',
 'BLL',
 'BAC',
 'BOH',
 'BXS',
 'BBG',
 'BAX',
 'AHC',
 'BDX',
 'BBBY',
 'BDC',
 'WRB',
 'BRK.A',
 'BBY',
 'BHP',
 'BIG',
 'ABMD',
 'BIO',
 'BKH',
 'BAF',
 'BOBE',
 'BA',
 'BOKF',
 'BAH',
 'BWA',
 'BPMP',
 'BDN',
 'CBM',
 'EAT',
 'BMY',
 'BR',
 'BRO',
 'BC',
 'BG',
 'CHRW',
 'CMA',
 'CBT',
 'COG',
 'CADE',
 'CPN',
 'CPT',
 'JCTCF',
 'CPB',
 'ACC',


In [111]:
Company_Name = []
Date = []
Earning_Call_list = []

In [3]:
Company_Name = []
Date = []
Earning_Call = []
NameText = []

In [42]:
url = 'https://seekingalpha.com/earnings/earnings-call-transcripts'
class MyOpener(FancyURLopener, object):
    version = choice(user_agents)
myopener = MyOpener()
html = myopener.open(url).read()
soup = BeautifulSoup(html, "lxml")

page_number = 1
target_page = 1000
while page_number <= target_page and ('Access to this page has been denied' not in html):
    print(page_number)
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    myopener = MyOpener()
    
    page_number += 1
    html = myopener.open(url).read()
    time.sleep(random.randint(2,15))
    soup = BeautifulSoup(html, "lxml")

    name_index = -1
    list_of_possible_companies = [x for x in list_of_companies if x in html]
    name_for_href = soup.find_all('a', class_ = 'dashboard-article-link')
    for name in soup.find_all('a', sasource = 'earnings-center-transcripts_symbol'):
        name_index += 1
        for company_name in list_of_possible_companies:
            if str(company_name) == name.text: #Checks if the 'name' iterator contains any of the listed companies
                #### Append Company Name ####
                Company_Name.append(company_name)

                #### Obatin and Append Date ####
                date = str(soup.find_all('div', class_ = 'article-desc')[name_index])
                date_start = date.find('<span class="bullet">•</span>')
                remaining = date[date_start:]
                date_end = remaining.find('<span>•</span>')
                date_as_str = remaining[len('<span class="bullet">•</span>'):date_end]
                Date.append(date_as_str.strip())

                #### Go to link, and obtain Call ####
                earning_call = ''
                inner_page_number = 1
                base_inner_page = 'https://seekingalpha.com' + name_for_href[name_index].get('href')
                inner_page = myopener.open(base_inner_page).read()
                #time.sleep(random.randint(5,15))
                inner_soup = BeautifulSoup(inner_page, 'lxml')

                # Iterate over each page, and obtain the call #
                for inner_text in inner_soup.find_all('p', class_ = re.compile('p p')):
                    earning_call += inner_text.text
                Earning_Call.append(earning_call)
                
        url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_number)


1
2
3
4
5
6
7
8
9


#### Dates are in a weird format:
Within 2017, they have a "Fri, Dec.  1,  9:56 PM" format. Every other year, they have a "Mar. 15, 2016,  8:03 PM" format

In [99]:
print(len(Earning_Call))

170


In [22]:
Company_Name

['GILD',
 'AEO',
 'JW.A',
 'MU',
 'BAC',
 'TOL',
 'JPM',
 'VZ',
 'AET',
 'WFC',
 'T',
 'S',
 'BNED',
 'MSFT',
 'EBAY',
 'CSCO',
 'ASNA',
 'CSCO',
 'TIME',
 'BIG',
 'EMR',
 'AFL',
 'F',
 'TD',
 'KR',
 'DCI',
 'CELG',
 'PVH',
 'SNPS',
 'V',
 'MSFT',
 'GLW',
 'BMY',
 'TIF',
 'AMD',
 'ADSK',
 'MRVL',
 'NUAN',
 'EBAY',
 'CSCO',
 'MU',
 'CBK',
 'CPRT',
 'HPE']

In [44]:
datadf = pd.DataFrame({'Company' : Company_Name_list, 'Date' : Date_list, 'Earning_Call' : Earning_Call_list})
datadf[datadf.Company == 'MKC']

,Company,Date,Earning_Call
12,MKC,"Thu, Sep. 28, 1:21 PM","McCormick & Company, Inc. (NYSE:MKC)Q3 2017 Re..."


In [ ]:
# Take care of the dates problem
datadf.Date = datadf.Date.str[:-10]
datadf.Date[datadf.Date.str[3] == ','] = datadf.Date.str[5:]
datadf.Date[datadf.Date.str.len() < 13] += ', 2017'
datadf.Date[datadf.Date.str.len() == 13] = datadf.Date.str[:3] + datadf.Date.str[4:]
datadf.Date[datadf.Date.str[4] == ' '] = datadf.Date.str[:4] + '0' + datadf.Date.str[5:] 

datadf.Date = pd.to_datetime(datadf.Date, format = '%m, %d, %Y')

Write the dataframe to a csv

In [24]:
datadf.to_csv('earning_calls_data.csv', encoding  = 'utf-8')